# Datos importantes:
 - pais q se puede sacar con la region lat y lon creo
 - leer que hace cada puto atributo
 - ya se guarda todo bien ns como (revisar todo el codigo)

### Función para descomprimir los datos del socket

In [35]:
def lzw_decompress(compressed_str):
    compressed = [ord(c) for c in compressed_str]
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError(f"Bad compressed k: {k}")
        result.append(entry)

        # Add w+entry[0] to the dictionary
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    return "".join(result)


### Funcion para limpiar

In [36]:
from datetime import datetime, timezone

def limpiar_datos(d):
    time_ns = d.get("time")

    try:
        # Convertir a segundos desde nanosegundos
        timestamp_sec = int(time_ns) / 1e9
        dt = datetime.fromtimestamp(timestamp_sec, tz=timezone.utc)
        fecha = dt.strftime("%Y-%m-%d")
        hora = dt.strftime("%H:%M:%S")
    except Exception as e:
        print("⚠️ Error al convertir timestamp:", time_ns, e)
        fecha = "1970-01-01"
        hora = "00:00:00"

    datos_limpios = {
        "time": int(time_ns),
        "lat": d.get("lat"),
        "lon": d.get("lon"),
        "alt": d.get("alt"),
        "pol": d.get("pol"),
        "mds": d.get("mds"),
        "mcg": d.get("mcg"),
        "status": d.get("status"),
        "region": d.get("region"),
        "delay": d.get("delay"),
        "lonc": d.get("lonc"),
        "latc": d.get("latc"),
        "fecha": fecha,
        "hora": hora
    }

    return datos_limpios


### Función para insertar los datos en MYSQL XAMPP

In [37]:
import mysql.connector

def guardar_rayo_mysql(data):
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="rayos"
        )
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO rayos (time, lat, lon, alt, pol, mds, mcg, status, region, delay, lonc, latc, fecha, hora)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            data.get("time"),
            data.get("lat"),
            data.get("lon"),
            data.get("alt"),
            data.get("pol"),
            data.get("mds"),
            data.get("mcg"),
            data.get("status"),
            data.get("region"),
            data.get("delay"),
            data.get("lonc"),
            data.get("latc"),
            data.get("fecha"),
            data.get("hora")
        ))
        conn.commit()
        cursor.close()
        conn.close()
    except Exception as e:
        print("❌ Error al guardar en MySQL:", e)


# Lectura del socket y guardado a MYSQL XAMPP


In [39]:
import json
import aiohttp
import asyncio

async def escuchar():
    url = "wss://ws2.blitzortung.org/"
    headers = {
        "Origin": "https://www.blitzortung.org",
        "User-Agent": "Mozilla/5.0",
    }

    session = aiohttp.ClientSession()
    contador_rayos = 0

    try:
        async with session.ws_connect(url, headers=headers) as ws:
            await ws.send_str('{"a":111}')  # handshake inicial

            async for msg in ws:
                if msg.type == aiohttp.WSMsgType.TEXT:
                    try:
                        raw = msg.data
                        texto = lzw_decompress(raw)
                        data = json.loads(texto)

                        rayo_principal = limpiar_datos(data)
                        if rayo_principal is not None:
                            guardar_rayo_mysql(rayo_principal)
                            contador_rayos += 1
                            print(f"✅ Rayo {contador_rayos} guardado: {rayo_principal['time']}")

                        if contador_rayos >= 10:
                            print("✅ Se han procesado 10 rayos. Finalizando...")
                            break
                    except Exception as e:
                        print("❌ Error al procesar mensaje:", e)
                elif msg.type in [aiohttp.WSMsgType.CLOSED, aiohttp.WSMsgType.ERROR]:
                    print("❌ WebSocket cerrado o con error")
                    break
    finally:
        await session.close()

# Ejecutar el bucle
await escuchar()


✅ Rayo 1 guardado: 1752701401897536300
✅ Rayo 2 guardado: 1752701400716851700
✅ Rayo 3 guardado: 1752701407549978400
✅ Rayo 4 guardado: 1752701407549986800
✅ Rayo 5 guardado: 1752701407550045700
✅ Rayo 6 guardado: 1752701407624181200
✅ Rayo 7 guardado: 1752701407718235100


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000020E8936B2B0>


✅ Rayo 8 guardado: 1752701408010718500
✅ Rayo 9 guardado: 1752701408010788000
✅ Rayo 10 guardado: 1752701408198030000
✅ Se han procesado 10 rayos. Finalizando...
